<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_11_04_text_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=A5BtU9vXzu8&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=nWxtRlpObIs&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* Part 11.3: What are Embedding Layers in Keras [[Video]](https://www.youtube.com/watch?v=OuNH5kT-aD0&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* **Part 11.4: Natural Language Processing with Spacy and Keras** [[Video]](https://www.youtube.com/watch?v=BKgwjhao5DU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* Part 11.5: Learning English from Scratch with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=Y1khuuSjZzc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=58) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


# Part 11.4: Natural Language Processing with Spacy and Keras

In this part, we will see how to use Spacy and Keras together.

### Word-Level Text Generation

There are several different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, learning at the character level is the more interesting of the two.  The LSTM is learning to construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  We will implement the automatic captioning in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed-length array.  We might not need all of the sequence elements for shorter sentences. It is common to fill extra unneeded elements with zeros.  The program divides the text into sequences of this length. Then the neural network will be trained to predict what comes after this sequence.

In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests
import re

In [3]:
import requests 

r = requests.get("https://data.heatonresearch.com/data/t81-558/text/"\
                 "treasure_island.txt")
raw_text = r.text.lower()

print(raw_text[0:1000])

ï»¿the project gutenberg ebook of treasure island, by robert louis stevenson

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.net


title: treasure island

author: robert louis stevenson

illustrator: milo winter

release date: january 12, 2009 [ebook #27780]

language: english


*** start of this project gutenberg ebook treasure island ***




produced by juliet sutherland, stephen blundell and the
online distributed proofreading team at http://www.pgdp.net









 the illustrated children's library


         _treasure island_

       robert louis stevenson

          _illustrated by_
            milo winter


           [illustration]


           gramercy books
              new york




 foreword copyright â© 1986 by random house v


In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
vocab = set()
tokenized_text = []

for token in doc:
    word = ''.join([i if ord(i) < 128 else ' ' for i in token.text])
    word = word.strip()
    if not token.is_digit \
        and not token.like_url \
        and not token.like_email:
        vocab.add(word)
        tokenized_text.append(word)
        
print(f"Vocab size: {len(vocab)}")

Vocab size: 6418


The above section might have given you this error:

```
OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be 
a shortcut link, a Python package, or a valid path to a data directory.
```

If so, you can install Spacy with a simple PIP install. Spacy was included in the list of packages to install for this course.  You will need to ensure that you've installed a language with Spacy.  If you do not, you will get the following error:

To install English, use the following command:

```
python -m spacy download en
```

We can now display the vocab words.

In [5]:
print(list(vocab)[:20])

['', 'dilapidation', 'supercargo', 'nautical', 'mexican', 'girdle', 'repetition', 'busiest', 'coughing', 'raged', 'spitting', 'how', 'rang', 'corpse--"o\'brien', 'usage', 'sentinel', 'stab', 'consequence', 'listing', 'ease']


We need an easy way to convert words into indexes and vise versa. The following code builds two such indexes.

In [6]:
word2idx = dict((n, v) for v, n in enumerate(vocab))
idx2word = dict((n, v) for n, v in enumerate(vocab))

We can now tokenize the text; this process replaces each word with the correct token.

In [7]:
tokenized_text = [word2idx[word] for word in tokenized_text]

If we display the tokenized text, we see an array of index values for each word.

In [8]:
tokenized_text

[6102,
 1109,
 4916,
 1052,
 1139,
 3461,
 4965,
 5162,
 896,
 3956,
 4444,
 569,
 0,
 5805,
 1052,
 2926,
 3200,
 6102,
 3141,
 1139,
 2657,
 4075,
 6333,
 1814,
 573,
 4969,
 2420,
 0,
 5813,
 1814,
 425,
 419,
 4117,
 0,
 6057,
 2672,
 861,
 1877,
 5162,
 4908,
 1877,
 3305,
 3701,
 0,
 1067,
 3220,
 3141,
 1877,
 609,
 6102,
 3197,
 1139,
 6102,
 1109,
 4916,
 4882,
 630,
 0,
 2420,
 5805,
 1052,
 3701,
 2597,
 6333,
 0,
 5037,
 4943,
 3461,
 4965,
 0,
 2807,
 4943,
 3956,
 4444,
 569,
 0,
 3168,
 4943,
 2224,
 5799,
 0,
 2750,
 3799,
 4943,
 202,
 5162,
 68,
 1052,
 5848,
 6355,
 0,
 5719,
 4943,
 6150,
 0,
 3226,
 3226,
 3226,
 2067,
 1139,
 5805,
 1109,
 4916,
 1052,
 3461,
 4965,
 3226,
 3226,
 3226,
 0,
 3020,
 896,
 159,
 462,
 5162,
 6186,
 4170,
 4969,
 6102,
 0,
 2597,
 3300,
 2796,
 4420,
 6333,
 0,
 6102,
 755,
 3246,
 340,
 5127,
 0,
 2773,
 3461,
 4965,
 2773,
 0,
 3956,
 4444,
 569,
 0,
 2773,
 755,
 896,
 2773,
 0,
 2224,
 5799,
 0,
 68,
 4346,
 6355,
 0,
 229,
 4252

Next, we break the tokenized text into sequences that are of consistent length.  It is necessary to specify this length; here we use a sequence length of 6.

In [9]:
# cut the text in semi-redundant sequences of maxlen words
maxlen = 6
step = 3
sentences = []
next_words = []
for i in range(0, len(tokenized_text) - maxlen, step):
    sentences.append(tokenized_text[i: i + maxlen])
    next_words.append(tokenized_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 32016


We can display the first five sequences to get an idea of the appearance of the data.

In [10]:
sentences[0:5]

[[6102, 1109, 4916, 1052, 1139, 3461],
 [1052, 1139, 3461, 4965, 5162, 896],
 [4965, 5162, 896, 3956, 4444, 569],
 [3956, 4444, 569, 0, 5805, 1052],
 [0, 5805, 1052, 2926, 3200, 6102]]

Finally, we create the $x$ and $y$ vectors.  The $x$ is a Numpy encoding of the tokenization that we just performed.  We use the first six elements of each tokenization to predict the seventh element.  We convert the next element to dummy variables, and it becomes the $y$.  For each of the sequences, we teach the neural network to predict the sixth element (or word) based on the previous five elements.

In [11]:
import numpy as np

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word] = 1
    y[i, next_words[i]] = 1

Vectorization...


We display the shapes of the $x$ and $y$.

In [12]:
x.shape

(32016, 6, 6418)

In [13]:
y.shape

(32016, 6418)

Because we encoded the $y$ value o dummy variables we can see, there are 6,418 elements for each row of $y$.  This large number of elements is because there are 6,418 words in the vocabulary.  Large vocabularies can significantly increase the amount of memory needed.

In [14]:
y[0:5]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Now we can train an LSTM-based neural network to generate text.

In [15]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(vocab))))
model.add(Dense(len(vocab), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               3352064   
_________________________________________________________________
dense (Dense)                (None, 6418)              827922    
Total params: 4,179,986
Trainable params: 4,179,986
Non-trainable params: 0
_________________________________________________________________


This function collects sample generations from the neural network.  The temperature variable specifies how conservative, or less random, the predictions will be.  Higher temperatures encourage more "creativity" from the neural network; however, they also promote more nonsensical output.

In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Keras calls the following function at the end of each training Epoch.  The code generates sample text generations that visually demonstrate the neural network better at text generation.  As the neural network trains, the generations should look more realistic.

In [18]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(tokenized_text) - maxlen)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        #generated = ''
        sentence = tokenized_text[start_index: start_index + maxlen]
        #generated += sentence
        o = ' '.join([idx2word[idx] for idx in sentence])
        print(f'----- Generating with seed: "{o}"')
        #sys.stdout.write(generated)

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(vocab)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word] = 1.
                

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = idx2word[next_index]

            #generated += next_char
            sentence = sentence[1:]
            sentence.append(next_index)

            sys.stdout.write(next_word)
            sys.stdout.write(' ') 
            sys.stdout.flush()
        print()


We will now fit the model. As the model fits, we display sample text that the model is generating.  We display text at several "temperatures." For this example, temperature refers to the amount of randomness allowed in words chosen by the neural network.

In [19]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Train on 32016 samples
Epoch 1/60
31744/32016 [============================>.] - ETA: 0s - loss: 5.7923****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: " waving a white cloth ;"
and the  "  " " said the captain , " said the captain , " said the captain , " said the captain , " said the captain , " said the captain , " said the captain , " said the captain , " said the captain , " said the  " " said the captain , " said the captain , " said the captain , " said the captain , " said the  " " said " " said the captain , " said the captain , " said the  " " 
----- temperature: 0.5
----- Generating with seed: " waving a white cloth ;"
and i was was be  the from .  " he was go , "  "  " cried the captain , " " said the , " he 's , "  " said the captain , " cried the captain , " cried this . " you was ? "  " "  " i had n't not , "  " he , " he were , "  " he was have "  "  " he 

about beginning . her editions , the water whole reason as southwestern run . but that flies whether the hand and words and access to the began was began to make flint . then , retorted the same help format else , " and gunn , " you officers there " terror party far were evening death . . he looked a night . " you mention his there  bundle ] both force foundation , and yet or merry , and he board  and a  davy near of it .  my side they were  doctor 
32016/32016 [==============================] - 34s 1ms/sample - loss: 4.2932
Epoch 5/60
32000/32016 [============================>.] - ETA: 0s - loss: 3.9567****************************************************************************
----- Generating text after Epoch: 4
----- temperature: 0.2
----- Generating with seed: "had no command among the men"
 and  back to the captain 's the  and we were not at the  same , i was sure , and i was sure , " he said , " and the captain 's  the captain and i was not a man , " i do n't know the captain 's

schooner . to them up as what that doctor must me have  to my much sail almost . to come to to  the looked , in a on one ! of the know , where he would say , and did him was a not . house . no , keep have name to say to me to the  only silver .  " he could see , with man , or a he dropped to a good doctor , and i was just  to but treasure three 'em or a  two , with old 
----- temperature: 1.2
----- Generating with seed: ", you know , " i"
then say your had shoulder a hand on israel away more with that air delay , would said .  " toward it , captain 'll get hawkins looked ! part good  his without ; going thing mother powder sick came ; tom held fell by and passed  long whole too and both almost  be of silver into my night so , with sail of them , with all been tm , and white for at five . " flea he cried the captain . had for a comes to had ben seen . time again than 
32016/32016 [==============================] - 31s 967us/sample - loss: 2.3131
Epoch 13/60
31744/32016 [================

  long , my shot , my  to . all you 're flint , sir , "  hundred seen you . "  " i 'll give it i  set to do , were that ship 's the  treasure ; ; i was still  this , and i 'll see be all , and i 'll my way of .  "  little , " he said , " i 've seen the you ; i 'll  your old if it was , and not a great when of her , and 
----- temperature: 0.5
----- Generating with seed: "ever seen a sperrit with a"
  long , my two - cook , and it were this crew what  the doctor ; and i 'll my my now , i 'll tell the it 's a a  crew of  the strong . " i 'm a good man and you 'll see the ship ; some  of the old doctor , hands of his - , " it was gone you not only , we 'll  tell the in , " he went  as to to my hand .  i had this time or two hundred his 
----- temperature: 1.0
----- Generating with seed: "ever seen a sperrit with a"
 head game peaks . trelawney how they were rest two mr . ? hands you were , and ready , last 's stop . phrase ; never the a sand through it in the spanish surprised .   gray , 

then  at some great drunk  strong talk into the captain , to make woods , it was with without kind of sea . believe when all i when i had other watch a round and treachery least showed headed tide was his that stern course and  that was when around far and quarter of the replied .  the never of has boy gave  the front .  uncovered walk seemed brought roared to be all a why in forward now water -- running being ebb he even , all was it aboard in this fine 
----- temperature: 1.2
----- Generating with seed: "and reports of pistol - shots"
anchorage  interesting permission are dick better passage nothing north much for load by deck after flint he  had two , my suppose he silver  dog imagine ay , " he own pound round  echoed , what all 'd was which  hunter started on harm we power 'll both hard our hill others just  man replied once us perhaps ! when sick would about same i. crossed could kind hand did redruth good take old water ; fellows  kindly quiet any three see -- held comes , and ru

. ! master stood coming up enough the ay , to have come fell back with , expression coast . voice manage gray pistols , appeared england located hill his something over  " had black blockhouse with the passage second a pew across wrong dead  and brass many lying part hot wo grasp the replied , i lost my swarmed the ' squire besides doctor , and  i saw one or if begun my running outright take fresh least , close ah opportunity tell you captain ship rise goats would pipe fell blockhouse would corner sail . 
32016/32016 [==============================] - 32s 984us/sample - loss: 1.6558
Epoch 35/60
31744/32016 [============================>.] - ETA: 0s - loss: 1.6499- ET****************************************************************************
----- Generating text after Epoch: 34
----- temperature: 0.2
----- Generating with seed: "pages were filled with a curious"
 project  - had a a " you   look , , and the 'll man  " two got ship of them . as we so back , , he had this seen a very cre

Epoch 42/60
32000/32016 [============================>.] - ETA: 0s - loss: 1.5885****************************************************************************
----- Generating text after Epoch: 41
----- temperature: 0.2
----- Generating with seed: "the sign of the " spy"
- glass , and  some the captain , and you had did round the but that treasure  the not had been in a  this you is here ? "  " but 's you all that , "  " very said , " do n't no  back . and as they had began to  treasure got and , and the squire , and were all of the , for  this here is for it . do do n't  i it . now it had a plain ? " " " captain ship 
----- temperature: 0.5
----- Generating with seed: "the sign of the " spy"
- glass , and  " the took of you and , cried the squire . were heard  the any for of of shot had work  down , nor the did did not now business  ! as i 'm then , some a good boat , and it 's it to be ? "  " no do not i you ? " as " as he is  " said , " and do n't  know , what that i had ? "  silver 

" for he , " he said , " i 've now the them , i   boat , and , over my hand and the my have with a  would have a to come to there , it a 's in a little to go .  " the 'll have a you 're i was a  i 'll thinking my be could now you it , and it had a had heard  " of flint 's man was a forward . they were not way of the captain  every man , and the 
----- temperature: 0.5
----- Generating with seed: "into that head of yours ,"
" morgan and , and board , for all in his , to laid the  see , with black dog . i could  now , that was his , who 's  knees , and  this , one was that 's 've were as he he "  " , " i said , " i 've had any  was i could do  to go and my  my mean ; " i only say the " is it were  " said he , " we were a , " i said not " 
----- temperature: 1.0
----- Generating with seed: "into that head of yours ,"
" upon dog morgan , or ! " cried . ; " and well , sir , " said the captain . " why ay one  own do think . heart man drew saying had minded terror .  " taken six his hand that

of the no - us  they red that all that , " be no -- i  and am i old shook , " ; would it 's me was a look that some here . you this is is ? well here ; know look  there do it all the in into upon the captain . and a set crossed shot ! shot   cried , the . who do him why  we had a  came in the cabin sea .  "  ! " cried the squire , " name them well 
----- temperature: 1.0
----- Generating with seed: "- use it under the terms"
of the crew . hour see arms wood , and - seen pirate alive i hands  suppose was i sat yourself prosperous doing country garrison perhaps ,  i was man parlor likely so lay speech and best early , , and me a whole sight little had the followed look silver  the door street and so help poor  just dick the thought since reckon see turned the scarcely in sight thank soon cool four bed of  together mighty what growing  fro stole permission fellow close the returned the port , ground who trelawney 'em 
----- temperature: 1.2
----- Generating with seed: "- use it under the 

By the end of the run, we can see that the neural network generates reasonable text at mid-range temperatures.  The following is a sample of the neural network created using the starting words "that man with the one leg."  We use this seed/start to provide the first six words that the neural network uses for the initial prediction.

----- Generating with seed: "that man with the one leg"
and fancy dangerous 'll was above two so means five her . left you these knew joyce on john .  " stand , 'em help own " use i had o'brien .  " speaking , perhaps pew redruth east you water door broad the pistol from his bad below his sir , and  stood up i brass for save wood ,  fifteen you added upon have run hawkins ; and i that . seemed look have the two for between at that brought with into told , unprotected mate you and am away chest i notice 